In [2]:
from hashlib import sha256
from math import gcd
from random import randint
from Crypto.Util.number import getPrime, inverse
import numpy as np

# Import lll
#from lll import lll

def key_gen():
    phi = 2
    e = 2
    while gcd(phi, e) != 1 :
        p = getPrime(1024)
        q = getPrime(1024)
        n = p*q
        phi = (p-1) * (q-1)
        e = 65537
    d = inverse(e, phi)
    return (e, d, n, p, q)

def sign(m, d, p, q, n, bug = False):
    dp = d % (p-1)
    dq = d % (q-1)
    h = int.from_bytes(sha256(m).digest(), byteorder = "big")

    if bug:
        sp = randint(1, p - 1)
    else:
        sp = pow(h, dp, p)

    sq = pow(h, dq, q)
    signature = (sp * q * pow(q, -1, p) + sq * p * pow(p, -1, q)) % n

    # PKCS#1 V1.5 signature
    prefix_sha256 = b'\x30\x31\x30\x0d\x06\x09\x60\x86\x48\x01\x65\x03\x04\x02\x01\x05\x00\x04\x20'
    pad_len = 256 - len(h.to_bytes(32, byteorder="big")) - len(prefix_sha256) - 3
    pad = b'\x00\x01' + b'\xff' * pad_len + b'\x00' + prefix_sha256 + h.to_bytes(32, byteorder="big")

    print("Signature (int):", signature)
    print("Signature (hex):", signature.to_bytes(256, byteorder="big").hex())
    print("Padding:", pad.hex())

    return signature, pad

def verify(m, s, e, n):
    decrypted = pow(s, e, n)
    decrypted_bytes = decrypted.to_bytes(256, byteorder="big")

    # Extraction du hash et comparaison
    h_calculated = sha256(m).digest()
    h_extracted = decrypted_bytes[-32:]

    print("Calculated hash from m:", h_calculated.hex())
    print("Extracted hash of pad :", h_extracted.hex())

    return h_calculated == h_extracted


def attack(s, e, n, pad):

    # Known part of the padding
    a = pad[:-32]
    n1 = (pow(s, e) - a) % n

    # Construct the matrix
    log_r = 256
    B = np.array([
        [-2**(2 * log_r), 2**log_r * n1, 0],
        [0, -2**log_r, n1],
        [0, 0, n]
    ])

    reduced_matrix = lll(B)

    # 3. Afficher la matrice après réduction
    print("Matrice réduite :")
    for row in reduced_matrix:
        print(row)

    v = reduced_matrix[0]

    r1 = v[2]  # On suppose que r_1 est dans la dernière colonne après réduction

    # Calcul de p à partir de la gcd
    from math import gcd
    p = gcd(n, n1 - r1)

    print(f"r1 : {r1}")
    print(f"p : {p}")


(e, d, n, p, q) = key_gen()
m = b"This message is signed with RSA-CRT!"
(s, pad) = sign(m, d, p, q, n)
# print("e = %s" % str(e))
# print("n = %s" %str(n))
# print("s = %s" %s)

print("Verify signature: ", verify(m, s, e, n))

print("-------------------")
print("Attack")
print("-------------------")
#attack(s, e, n, pad)

Signature (int): 5951000455699118081374757397968414235999543737511217727006072272585466432954148415719973927547240679803516747880774126999917859247565099501305314467275753745934256216032190551905563832266741026417052611226236205226839805713935237374792632230009701493076755829358366166368102985740751981246341577858535582982713139017575468673143438879276249549627890946547925813033592251463256431648858792180451846649258005119867191820342988104914248827226297170568995934208707031435887344832258854147839053402705733787156832265857972409214542792268247937432732614615373370675243429293109636765785016163820932469602730846533169376628
Signature (hex): 2f2418d8bc914eef91ca30d78effde461be103811a37cb698823694c372de7d0e20a5ed895195f4bde2e7462ef3b7db5faa9035ccc0c44f0585fec2355f83e677461d551481f3a4abfcd6da0358ab95b870a5cce53f42f12ba37c69604a110a13a64809d126e3d96eb0506c9b913776fa1026015895608fe666d9c985e357cf3c9d11f2bb1e94dea52f76f0d71561d8a872024cfa3cd04deca5e0eca96ebcb8ea76ad3927303275e1cb91f97597b2

KeyboardInterrupt: 